# Initiation and maintenance splitter

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/home/jupyter/charliemacuject/research_papers/data/master_allstats.csv')
# ndf = ndf.rename(columns={"Patient::PatientBCID": "ID", "Patient::PatientGender": "gender", "Current age in 2020": "age"})
df.head()

,Unnamed: 0,gender,age,CreatedDate,ExamGraph,NextTime,DaysFirst,actual_time,adherence_factor,running_ad,st_dev,prev_vision,mean_vision,std_vision,ID
0,118,Male,91,2017-05-09,0.333333,8.0,104,6.0,1.00,1.25,0.0000,0.333333,0.30,0.0981,0
1,119,Male,91,2017-08-08,0.333333,12.0,195,13.0,1.62,1.12,0.1250,0.333333,0.31,0.0862,0
2,120,Male,91,2017-10-31,0.333333,12.0,279,12.0,1.00,1.29,0.2547,0.333333,0.31,0.0777,0
3,121,Male,91,2018-01-23,0.333333,12.0,363,12.0,1.00,1.22,0.2538,0.333333,0.32,0.0714,0
4,122,Male,91,2018-04-24,0.333333,12.0,454,13.0,1.08,1.17,0.2431,0.333333,0.32,0.0663,0


In [3]:
df.shape

(6637, 15)

## Check what some intervals look like

In [4]:
intervals = check_id['actual_time']
intervals

NameError: name 'check_id' is not defined

## Use biggest jump to determine where cutoff should be

In [5]:
# get the test dataframe: one patient
test_df = df[df['ID'] == 2]

### Doctest for biggest jump

In [6]:
# this function loops through a list to find the biggest jump 
def biggest_jump(interval_list):
    '''
    >>> biggest_jump(test_interval_list)
    14
    '''
    diff, index = 0, 0
    total = len(interval_list)
    for i in range(len(interval_list)-1):
        if interval_list[i+1] - interval_list[i] > diff and (i/total)> 0.2:
            diff = interval_list[i+1] - interval_list[i]
            index = i+1
        else:
            i += 1
    return index

In [7]:
import doctest
test_interval_list = [4,4,4,4,4,5,5,5,5,5,6,6,6,6,10,10,10,12,11,12]
doctest.testmod()

TestResults(failed=0, attempted=1)

In [8]:
intervals = check_id['actual_time'].tolist()
index = biggest_jump(intervals)
index

NameError: name 'check_id' is not defined

### Split the individual dataframe into initiation and maintenance, based on above index

In [9]:
test_df_initiation = test_df.iloc[:index,:]
test_df_maintenance = test_df.iloc[index:,:]
test_df_initiation

NameError: name 'index' is not defined

## Split each patient's visits and add them to initiation or maintenance

In [10]:
# function to combine all of the above operations
def init_maint_splitter(patient_df):
    intervals = patient_df['actual_time'].tolist()
    index = biggest_jump(intervals)
    patient_df_initiation = patient_df.iloc[:index,:]
    patient_df_maintenance = patient_df.iloc[index:,:]
    return patient_df_initiation, patient_df_maintenance

In [11]:
test_init, test_maint = init_maint_splitter(test_df)
# test_init

In [14]:
# function to do the split to each patient, then concat
def master_dataframe():
    df = pd.read_csv('/home/jupyter/charliemacuject/research_papers/data/master_allstats.csv')
    id_list = df["ID"].unique()
    frames_init = []
    frames_maint = []
    for i in range(len(id_list)):
        patient_df = df[df['ID'] == i]
        try:
            init, maint = init_maint_splitter(patient_df)
            frames_init.append(init)
            frames_maint.append(maint)
        except:
            i += 1
    initiation_df = pd.concat(frames_init)
    maintenance_df = pd.concat(frames_maint)
    return initiation_df, maintenance_df

In [15]:
initiation_df, maintenance_df = master_dataframe()
initiation_df.shape, maintenance_df.shape

((3529, 15), (2623, 15))

In [16]:
initiation_df

,Unnamed: 0,gender,age,CreatedDate,ExamGraph,NextTime,DaysFirst,actual_time,adherence_factor,running_ad,st_dev,prev_vision,mean_vision,std_vision,ID
0,118,Male,91,2017-05-09,0.333333,8.0,104,6.0,1.00,1.25,0.0000,0.333333,0.30,0.0981,0
1,119,Male,91,2017-08-08,0.333333,12.0,195,13.0,1.62,1.12,0.1250,0.333333,0.31,0.0862,0
2,120,Male,91,2017-10-31,0.333333,12.0,279,12.0,1.00,1.29,0.2547,0.333333,0.31,0.0777,0
3,121,Male,91,2018-01-23,0.333333,12.0,363,12.0,1.00,1.22,0.2538,0.333333,0.32,0.0714,0
4,122,Male,91,2018-04-24,0.333333,12.0,454,13.0,1.08,1.17,0.2431,0.333333,0.32,0.0663,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6145,50957,Female,87,2020-02-04,0.500000,4.0,1378,4.0,1.00,0.92,0.2379,0.666667,0.73,0.1525,469
6146,50958,Female,87,2020-03-10,0.800000,4.0,1413,5.0,1.25,0.93,0.2336,0.500000,0.73,0.1555,469
6147,50959,Female,87,2020-05-05,0.800000,4.0,1469,8.0,2.00,0.94,0.2373,0.800000,0.73,0.1536,469
6148,50960,Female,87,2020-06-02,0.800000,4.0,1497,4.0,1.00,0.98,0.3071,0.800000,0.73,0.1518,469


## Save dataframes to data folders

In [18]:
initiation_df.to_csv('/home/jupyter/charliemacuject/research_papers/data/initiation_df.csv')
maintenance_df.to_csv('/home/jupyter/charliemacuject/research_papers/data/maintenance_df.csv')